In [36]:
import tensorflow as tf
import numpy as np
import cv2
import os


In [37]:
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

In [38]:
def extract_features_deep(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (300, 300))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    image = np.expand_dims(image, axis=0)
    
    # Extract features
    features = model.predict(image)
    
    # Flatten the feature maps
    features = features.flatten()
    
    return features

In [2]:
def extract_features(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Create ORB detector
    orb = cv2.ORB_create()
    
    # Find the keypoints and descriptors
    keypoints, descriptors = orb.detectAndCompute(image, None)
    
    return descriptors

In [3]:
def match_images(descriptor1, descriptor2):
    # Create a Brute Force Matcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    # Match descriptors
    matches = bf.match(descriptor1, descriptor2)
    
    # Sort the matches based on distance (lower distance is better)
    matches = sorted(matches, key=lambda x: x.distance)
    
    return matches

In [13]:
def search_database(query_image_path, database_folder, threshold=100):
    # Extract features from the query image
    query_descriptors = extract_features(query_image_path)
    
    best_match = None
    best_score = float('inf')
    
    for image_file in os.listdir(database_folder):
        image_path = os.path.join(database_folder, image_file)
        
        # Extract features from the current database image
        db_descriptors = extract_features(image_path)
        
        if db_descriptors is None or query_descriptors is None:
            continue  # skip if no features are found in either image
        
        # Match the query image descriptors with the database image descriptors
        matches = match_images(query_descriptors, db_descriptors)
        
        # Score based on the number of good matches (sum of distances)
        score = sum([m.distance for m in matches])
        
        if score < best_score:
            best_score = score
            best_match = image_file
    
    # Check if the best match score exceeds the threshold
    print(best_score)
    if best_score > threshold:
        return None  # No match found
    
    return best_match

In [39]:
def search_database_deep(query_image_path, database_folder, threshold=0.5):
    # Extract deep features for the query image
    query_features = extract_features_deep(query_image_path)
    
    best_match = None
    best_score = float(0)
    
    for image_file in os.listdir(database_folder):
        image_path = os.path.join(database_folder, image_file)
        
        # Extract deep features for the current database image
        db_features = extract_features_deep(image_path)
        
        # Calculate the cosine similarity between the feature vectors
        score = np.dot(query_features, db_features) / (np.linalg.norm(query_features) * np.linalg.norm(db_features))
        print(score)
        if score > best_score:
            best_score = score
            best_match = image_file
    print(best_score)
    # Check if the best match score exceeds the threshold
    if best_score > threshold:
        return best_match
    else:
        return None

In [43]:
# Path to the captured image
query_image_path = 'test_images/1.png'

# Path to the folder where your database images are stored
database_folder = 'sample_images'

# Find the best match in the database
best_match = search_database_deep(query_image_path, database_folder, threshold=0.2)

if best_match:
    print(f"Best matching image: {best_match}")
else:
    print("No similar image found in the database.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
0.1398467
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
0.20445436
0.20445436
Best matching image: 1.png
